In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import mysql.connector

app = Flask(__name__)

# Configure MySQL connection
mysql_db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="flight_booking"
)

# Create a cursor object to interact with the database
db_cursor = mysql_db.cursor()

# Define routes and their corresponding actions
@app.route('/')
def index():
    # Display the home page with flight search form
    return render_template('index.html')

@app.route('/search', methods=['POST'])
def search_flights():
    # Retrieve user input from the search form
    source = request.form['source']
    destination = request.form['destination']
    date = request.form['date']

    # Query the database to find matching flights
    query = "SELECT * FROM flights WHERE source=%s AND destination=%s AND date=%s"
    db_cursor.execute(query, (source, destination, date))
    flights = db_cursor.fetchall()

    # Render the search results template with the matching flights
    return render_template('search_results.html', flights=flights)

@app.route('/book', methods=['POST'])
def book_flight():
    # Retrieve flight and user information from the booking form
    flight_id = request.form['flight_id']
    name = request.form['name']
    email = request.form['email']

    # Insert the booking details into the database
    query = "INSERT INTO bookings (flight_id, name, email) VALUES (%s, %s, %s)"
    db_cursor.execute(query, (flight_id, name, email))
    mysql_db.commit()

    # Redirect to a booking confirmation page
    return redirect(url_for('confirmation'))

@app.route('/confirmation')
def confirmation():
    # Display a booking confirmation message
    return render_template('confirmation.html')

if __name__ == '__main__':
    app.run(debug=True)
